In [ ]:
%run "/Shared/Common Utilities/helper_utils"

In [ ]:
# Import the Data pretty printer - pprint library to help print data structures in
# a readable, pretty way

import pprint

#  To utilize the configuration values in the helper_utils Notebook

config = get_config()

In [ ]:
file_location = "wasbs://sas-extracts@cctdash0000prd0504blob.blob.core.windows.net/Weekly .xlsx"

In [ ]:
#write explanation here
# table_name = temp_table_source = config["cct_db_temp_schema"] + ".temp_test_delme"

#any point? 
pp = pprint.PrettyPrinter(indent=4)

#comes from  helper_utils.py
moh_hosp_weekly_icu_df = spark.read.format("com.crealytics.spark.excel").options(header=True,inferSchema=True).load(file_location) 


moh_hosp_weekly_icu_df.createOrReplaceTempView("moh_hosp_weekly_icu_df")

query = "SELECT * from moh_hosp_weekly_icu"
#display(spark.sql(query))
    
#create_table_as_query_and_assign_perms("weeklyICU_mohsas_df", query) # fn coming from  helper_utils.py => /Shared/Common Utilities/helper_utils
    
#spark.sql(query)

In [ ]:
moh_hosp_weekly_icu_df.printSchema()

root
 |-- Week: string (nullable = true)
 |-- Available baseline beds (avg): double (nullable = true)
 |-- Adult baseline occupancy: double (nullable = true)
 |-- Vented adult baseline occupancy: double (nullable = true)
 |-- Adult baseline beds (avg): double (nullable = true)
 |-- Vented adult baseline beds (avg): double (nullable = true)
 |-- Adult baseline patients (avg): double (nullable = true)
 |-- Vented adult baseline patients (avg): double (nullable = true)



In [ ]:
#renames columns

moh_hosp_weekly_icu_df=make_col_names_safe(moh_hosp_weekly_icu_df)

In [ ]:
moh_hosp_icu_weekly_table_source = config["cct_db_temp_schema"] + ".temp_test"
print(moh_hosp_icu_weekly_table_source)
spark.sql(f"DROP TABLE IF EXISTS hive_metastore.default.moh_hosp_icu_weekly_table_source")
# spark.sql(f"DROP TABLE IF EXISTS {temp_table_source}") 
display(moh_hosp_weekly_icu_df)
moh_hosp_weekly_icu_df.write.saveAsTable("moh_hosp_icu_weekly_table_source")


 #hive_metastore.default.temp_table_source

# #moh_hosp_weekly_icu_df = make_col_names_safe(moh_hosp_weekly_icu_df)
# moh_hosp_weekly_icu_df.write.saveAsTable(temp_table_source)

#moh_hosp_weekly_icu_df.withColumnRenamed("week","reported_week").write.saveAsTable(moh_hosp_icu_weekly_source)

In [ ]:
test_file = config["cct_db_schema"] + ".public_test"
print(test_file)

# prevent table duplication
spark.sql(f"DROP TABLE  IF EXISTS {test_file}")
# moh_hosp_weekly_icu_df.write.saveAsTable(test_file)
display(test_file)

development.public.public_test
'development.public.public_test'

In [ ]:
import re
from pyspark.sql.functions import col, to_date

def second_dateFix(str):
    lowered = str.lower().replace(" ", '')

    second_type_of_date = re.compile("[0-9]{4}[a-z]{3}[0-9]{6}") #2020jan022021 (R)  #2020 Week20 May23 2021 (K)
    match = second_type_of_date.match(lowered)
    
    if match is not None:
        fp = re.sub("^[0-9]{4}", '', lowered)
        arr = re.findall('(^[a-z]{3})([0-9]{2})([0-9]{4})', fp)
        return arr[0][2]+arr[0][0]+arr[0][1]
        # return arr
    else:
        return str

#lowered = str.lower().replace(" ", '')


def dateFix(str):
    # lowered = str.lower().replace(" ", '')

    return re.sub("week[0-9][0-9]", '', lowered)




In [ ]:
# moh_hosp_weekly_icu_df.select("week").show()

a = moh_hosp_weekly_icu_df.select("week")
# display(a)

from pyspark.sql.functions import when, regexp_replace, col, concat, to_date
# from pyspark.sql.functions import functions as f
import pyspark.sql.functions as f

b = a.withColumn('week', when(a.week.contains("Week"),regexp_replace(a.week,'Week','')))
b = b.withColumn('week', when(b.week.contains(" "),regexp_replace(b.week,' ','-')))
split_col = f.split(b['week'], '-')

b = b.withColumn('year', split_col.getItem(0))\
     .withColumn('week_num', split_col.getItem(1))\
     .withColumn('Month_date', split_col.getItem(2))\
     .withColumn('Year_latest', split_col.getItem(3))

b = b.withColumn('Month', concat(b.Month_date.substr(0, 3)))\
     .withColumn('Date', concat(b.Month_date.substr(4, 5)))
b = b.select('Date','Month','year',when(b.Year_latest.isNull(), b.year).otherwise(b.Year_latest).alias('Year_latest'))
# display(b)

b = b.withColumn('Date', f.concat(f.col('Date'), f.lit(" "), f.col('Month'), f.lit(" "), f.col('Year_latest')))
b = b.drop("year","week","week_num","Month_date","Month","Year_latest")

b = b.select(col("Date"), to_date(col("Date"), "dd MMM yyyy").alias("Transformed_Date")).drop("Date")
display(b)  ######transformed date

display(a)   ##original date


In [ ]:
moh_hosp_weekly_icu_df 